# Análisis de Datos de Caesars Entertainment
## Análisis de Check-ins, Eventos Especiales y Tipos de Cliente

Este notebook contiene un análisis completo de los datos de Caesars Entertainment, incluyendo:
- Análisis descriptivo de check-ins
- Visualizaciones de tendencias y patrones
- Modelos de regresión para entender factores de demanda
- Evaluación del impacto de eventos especiales

# 1. Carga y Preparación de Datos

Primero cargaremos las librerías necesarias y prepararemos los datos para el análisis.

In [ ]:
# Cargar librerías necesarias
library(readxl)      # Para leer archivos Excel
library(dplyr)       # Para manipulación de datos
library(tidyr)       # Para limpieza de datos
library(ggplot2)     # Para visualizaciones
library(lubridate)   # Para manejo de fechas
library(broom)       # Para trabajar con modelos estadísticos

# Leer los datos
caesars_data <- read_excel("Caesars.XLS.xlsx")

# Examinar la estructura de los datos
str(caesars_data)
summary(caesars_data)

# Convertir la columna de fecha a formato Date
caesars_data <- caesars_data %>%
  mutate(Date = as.Date(Date))

# Asegurar que las variables dummy sean numéricas (0 o 1)
dummy_vars <- c("SuperBowl", "Valentines", "NY", "Easter", "LaborDay", "Xmas")
caesars_data <- caesars_data %>%
  mutate(across(all_of(dummy_vars), as.numeric))

# Verificar y manejar valores faltantes
sum(is.na(caesars_data)) # Contar valores faltantes
caesars_data <- na.omit(caesars_data) # Eliminar filas con valores faltantes

# Mostrar las primeras filas del dataset limpio
head(caesars_data)

# 2. Estadísticas Descriptivas

A continuación, generaremos tablas descriptivas para entender mejor los patrones en los datos.

In [ ]:
# Estadísticas descriptivas de check-ins
checkins_stats <- caesars_data %>%
  summarise(
    Promedio = mean(Checkins),
    Min = min(Checkins),
    Max = max(Checkins),
    Desv_Est = sd(Checkins)
  )
print("Estadísticas de Check-ins:")
print(checkins_stats)

# Promedio de check-ins por mes
checkins_por_mes <- caesars_data %>%
  mutate(Mes = month(Date, label = TRUE)) %>%
  group_by(Mes) %>%
  summarise(
    Promedio_Checkins = mean(Checkins),
    N = n()
  )
print("\nPromedio de Check-ins por Mes:")
print(checkins_por_mes)

# Promedio de check-ins por día de la semana
checkins_por_dia <- caesars_data %>%
  mutate(DiaSemana = wday(Date, label = TRUE)) %>%
  group_by(DiaSemana) %>%
  summarise(
    Promedio_Checkins = mean(Checkins),
    N = n()
  )
print("\nPromedio de Check-ins por Día de la Semana:")
print(checkins_por_dia)

# Resumen general de variables importantes
print("\nResumen de Variables Importantes:")
summary(caesars_data[c("Checkins", "FIT_ADR", "Casino", "Group", "SE")])

# 3. Visualizaciones

Crearemos una serie de gráficos para visualizar patrones en los datos.

In [ ]:
# Gráfico de línea temporal de check-ins
ggplot(caesars_data, aes(x = Date, y = Checkins)) +
  geom_line(color = "blue") +
  geom_smooth(method = "loess", color = "red") +
  labs(title = "Evolución de Check-ins a lo Largo del Tiempo",
       x = "Fecha",
       y = "Número de Check-ins") +
  theme_minimal()

# Gráfico de barras de promedio de check-ins por mes
caesars_data %>%
  mutate(Mes = month(Date, label = TRUE)) %>%
  group_by(Mes) %>%
  summarise(Promedio_Checkins = mean(Checkins)) %>%
  ggplot(aes(x = Mes, y = Promedio_Checkins)) +
  geom_bar(stat = "identity", fill = "skyblue") +
  labs(title = "Promedio de Check-ins por Mes",
       x = "Mes",
       y = "Promedio de Check-ins") +
  theme_minimal()

# Gráfico comparativo de check-ins en días con eventos vs sin eventos
eventos_comparacion <- caesars_data %>%
  mutate(Tiene_Evento = if_else(SuperBowl == 1 | Valentines == 1 | NY == 1 |
                               Easter == 1 | LaborDay == 1 | Xmas == 1,
                               "Con Evento", "Sin Evento")) %>%
  group_by(Tiene_Evento) %>%
  summarise(Promedio_Checkins = mean(Checkins))

ggplot(eventos_comparacion, aes(x = Tiene_Evento, y = Promedio_Checkins, fill = Tiene_Evento)) +
  geom_bar(stat = "identity") +
  labs(title = "Comparación de Check-ins: Días con Eventos vs Sin Eventos",
       x = "Tipo de Día",
       y = "Promedio de Check-ins") +
  theme_minimal() +
  scale_fill_manual(values = c("Con Evento" = "darkgreen", "Sin Evento" = "lightgreen"))

# 4. Modelo de Regresión Lineal Simple

Analizaremos la relación entre el precio promedio (FIT_ADR) y el número de check-ins.

In [ ]:
# Modelo de regresión lineal simple
modelo1 <- lm(Checkins ~ FIT_ADR, data = caesars_data)

# Mostrar resumen del modelo
summary(modelo1)

# Visualizar la relación
ggplot(caesars_data, aes(x = FIT_ADR, y = Checkins)) +
  geom_point(alpha = 0.5) +
  geom_smooth(method = "lm", color = "blue") +
  labs(title = "Relación entre Precio Promedio y Check-ins",
       x = "Precio Promedio (FIT_ADR)",
       y = "Número de Check-ins") +
  theme_minimal()

# Interpretación del coeficiente
cat("\nInterpretación del coeficiente de FIT_ADR:\n")
cat("Un aumento de 1 unidad en el precio promedio (FIT_ADR) está asociado con un cambio de",
    round(coef(modelo1)[2], 2),
    "unidades en el número de check-ins, manteniendo todo lo demás constante.\n")

# 5. Modelo de Regresión Lineal Múltiple

Ahora incluiremos múltiples variables explicativas, incluyendo las variables dummy de eventos especiales.

In [ ]:
# Añadir variable de día de la semana
caesars_data <- caesars_data %>%
  mutate(DiaSemana = as.factor(wday(Date)))

# Modelo de regresión múltiple
modelo2 <- lm(Checkins ~ FIT_ADR + Casino + Group + SE + SuperBowl + 
              Valentines + NY + Easter + LaborDay + Xmas + DiaSemana,
              data = caesars_data)

# Mostrar resumen del modelo
summary(modelo2)

# Identificar variables significativas
coef_summary <- tidy(modelo2) %>%
  mutate(
    significativo = ifelse(p.value < 0.05, "Sí", "No")
  )

# Mostrar variables significativas
cat("\nVariables significativas (p < 0.05):\n")
print(coef_summary %>%
      filter(significativo == "Sí") %>%
      select(term, estimate, p.value))

# 6. Modelo con Tratamientos y Controles

Analizaremos el efecto conjunto de los eventos especiales como tratamiento.

In [ ]:
# Crear variable de tratamiento
caesars_data <- caesars_data %>%
  mutate(Treatment = as.numeric(SuperBowl == 1 | Valentines == 1 | NY == 1 |
                               Easter == 1 | LaborDay == 1 | Xmas == 1))

# Modelo con tratamiento
modelo3 <- lm(Checkins ~ Treatment + FIT_ADR + Casino + Group + SE,
              data = caesars_data)

# Mostrar resumen del modelo
summary(modelo3)

# Visualizar el efecto del tratamiento
ggplot(caesars_data, aes(x = factor(Treatment), y = Checkins, fill = factor(Treatment))) +
  geom_boxplot() +
  labs(title = "Distribución de Check-ins según Tratamiento",
       x = "Tratamiento (0 = Control, 1 = Evento Especial)",
       y = "Número de Check-ins") +
  scale_fill_manual(values = c("0" = "lightblue", "1" = "darkblue")) +
  theme_minimal() +
  theme(legend.position = "none")

# Calcular el efecto promedio del tratamiento
treatment_effect <- caesars_data %>%
  group_by(Treatment) %>%
  summarise(
    Promedio_Checkins = mean(Checkins),
    N = n()
  )

print("Efecto promedio del tratamiento:")
print(treatment_effect)

# 7. Comparación de Modelos

Compararemos el desempeño de los tres modelos utilizando diferentes métricas.

In [ ]:
# Crear función para extraer métricas
get_model_metrics <- function(model) {
  list(
    R2 = summary(model)$r.squared,
    R2_adj = summary(model)$adj.r.squared,
    AIC = AIC(model),
    BIC = BIC(model),
    RMSE = sqrt(mean(model$residuals^2))
  )
}

# Obtener métricas para cada modelo
metrics1 <- get_model_metrics(modelo1)
metrics2 <- get_model_metrics(modelo2)
metrics3 <- get_model_metrics(modelo3)

# Crear data frame de comparación
comparacion_modelos <- data.frame(
  Modelo = c("Modelo Simple", "Modelo Múltiple", "Modelo Tratamiento"),
  R2 = c(metrics1$R2, metrics2$R2, metrics3$R2),
  R2_Ajustado = c(metrics1$R2_adj, metrics2$R2_adj, metrics3$R2_adj),
  AIC = c(metrics1$AIC, metrics2$AIC, metrics3$AIC),
  BIC = c(metrics1$BIC, metrics2$BIC, metrics3$BIC),
  RMSE = c(metrics1$RMSE, metrics2$RMSE, metrics3$RMSE)
)

# Mostrar tabla de comparación
print("Comparación de Modelos:")
print(comparacion_modelos)

# Visualizar R² ajustado
ggplot(comparacion_modelos, aes(x = Modelo, y = R2_Ajustado, fill = Modelo)) +
  geom_bar(stat = "identity") +
  labs(title = "Comparación de R² Ajustado entre Modelos",
       x = "Modelo",
       y = "R² Ajustado") +
  theme_minimal() +
  theme(axis.text.x = element_text(angle = 45, hjust = 1))

# 8. Conclusiones Automáticas

Generaremos interpretaciones automáticas de los principales hallazgos.

In [ ]:
# Generar conclusiones automáticas
cat("\nCONCLUSIONES PRINCIPALES:\n\n")

# Efecto del precio
precio_efecto <- coef(modelo1)[2]
cat("1. EFECTO DEL PRECIO:\n")
if (precio_efecto < 0) {
  cat("- El precio tiene un efecto negativo en los check-ins: por cada unidad que aumenta el precio,",
      "los check-ins disminuyen en", abs(round(precio_efecto, 2)), "unidades.\n")
} else {
  cat("- El precio tiene un efecto positivo en los check-ins: por cada unidad que aumenta el precio,",
      "los check-ins aumentan en", round(precio_efecto, 2), "unidades.\n")
}

# Eventos más significativos
eventos_coef <- coef(modelo2)[c("SuperBowl", "Valentines", "NY", "Easter", "LaborDay", "Xmas")]
evento_max <- names(which.max(abs(eventos_coef)))
cat("\n2. IMPACTO DE EVENTOS:\n")
cat("- El evento con mayor impacto es:", evento_max,
    "con un efecto de", round(eventos_coef[evento_max], 2), "check-ins\n")

# Tipo de cliente
tipos_cliente <- c("Casino", "Group", "SE")
tipos_coef <- coef(modelo2)[tipos_cliente]
tipo_max <- names(which.max(abs(tipos_coef)))
cat("\n3. TIPO DE CLIENTE:\n")
cat("- El segmento de cliente con mayor impacto es:", tipo_max,
    "con un efecto de", round(tipos_coef[tipo_max], 2), "check-ins\n")

# Mejor modelo
mejores_r2 <- comparacion_modelos$R2_Ajustado
mejor_modelo <- comparacion_modelos$Modelo[which.max(mejores_r2)]
cat("\n4. MEJOR MODELO:\n")
cat("- El", mejor_modelo, "es el que mejor explica la variación en los check-ins,",
    "con un R² ajustado de", round(max(mejores_r2), 3), "\n")